$$
\newcommand{\pd}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bw}[2]{\mathbf{w}_{#1}^{\mathrm{#2}}}
\newcommand{\dbp}[2]{\delta\mathbf{p}_{#1}^{\mathrm{#2}}}
$$
## Lorenzモデル

Lorenz (1963)のモデル（以下Lorenz-63）は熱対流を理想化したモデルで，パラメタ次第でカオスにふるまいます。Lorenz-63は次の3変数の常微分方程式で表されます。

$$
\begin{align}
\dot{X} &= -\sigma X + \sigma Y \\
\dot{Y} &= -XZ + rX -Y \\ \label{l63}\tag{1}
\dot{Z} &= XY - \beta Z
\end{align}
$$

パラメタ$\sigma$, $r$, $\beta$ はそれぞれPrandtl数，Rayleigh数，アスペクト比を表しています。
予報変数$X, Y, Z$はそれぞれ無次元化された対流の強さ，最大温度差，対流に伴う成層の変化を表すものです。
左辺の変数の上の$\cdot$は時間微分$\mathrm{d}/\mathrm{d}t$を表す記号です。

　Lorenz-63を使って，まず変分法を用いて観測を同化して最適な初期値を求めてみましょう。次にニューラルネットワークを使って教師データからモデルを推定します。


### 変分法データ同化

モデルの状態がデータとの乖離の尺度は，評価函数やコスト函数と呼ばれています。
ここでは，各タイムステップにおける二条誤差の和で表します。

$$
J = \frac{1}{2}\sum_{k=0}^{n}(\bw{k}{f}-\bw{k}{o})^T(\bw{k}{f}-\bw{k}{o})\label{cost}\tag{2}
$$

上付きの$T$は転置を表します。ベクトルは縦に要素を並べた列ベクトルなので，その転置は行ベクトルです。
行ベクトルと列ベクトルの積は要素の二乗和つまり内積を表します。
各要素はタイムステップ$k$で予報$\mathrm{f}$と観測$\mathrm{o}$との差です。

最適な初期値に対しては，コスト函数は最小となります。
高校の数学で習った，2次函数の最小（最大）値は微分が0と同様です。
上のコスト函数は2次なので，勾配∇J = 0になるところを探せばよいということになります。
勾配はどのようにして求めればよいのでしょうか。
まず，式(\ref{cost})を微分して

$$
\delta J(\bw{0}{f}) = \sum_k(\bw{k}{f}-\bw{k}{o})^T\delta\bw{k}{f} \label{J'}\tag{3}
$$

が得られます。一方摂動 $\bw{0}{f}$ が小さいときコスト函数は

$$
\delta J(\bw{0}{f}) = J(\bw{0}{f}+\delta\bw{0}{f}) - J(\bw{0}{f}) \approx \left[\nabla J(\bw{0}{f})\right]^T\delta\bw{0}{f} \label{dJ}\tag{4}
$$

と書けます。(\ref{J'})と(\ref{dJ})より

$$
\left[\nabla J(\bw{0}{f})\right]^T\delta\bw{0}{f} = \sum_k(\bw{k}{f}-\bw{k}{o})^T\delta\bw{k}{f} \label{ww}\tag{5}
$$

を得ます。(\ref{ww})を解いて勾配を求めるには，摂動の時間発展を知る必要があります。非線型モデルは記号$\mathbf{N}$を用いて

$$
\bw{k}{f} = N\left(\bw{k-1}{f}\right) \label{N}\tag{6}
$$

と表されます。これを基本場の周りで線型化すると擾乱の時間発展は

$$
\delta\bw{k}{f} = \pd{\mathbf{N}\left(\bw{k-1}{f}\right)}{\bw{k-1}{f}}\delta\bw{k-1}{f} = \mathbf{L}\left(\bw{k-1}{f}\right)\delta\bw{k-1}{f} \label{defL}\tag{7}
$$

と表すことができます。
$\mathbf{L}\left(\bw{k-1}{f}\right)$ は接線型演算子と呼ばれ摂動の時間発展を表しています。
接線型演算子を繰り返し用いることにより $\delta\bw{k}{f}$ と$\delta\bw{0}{f}$の関係を次のように表すことができます。

$$
\delta\bw{k}{f} = \mathbf{L}\left(\bw{k-1}{f}\right)\mathbf{L}\left(\bw{k-2}{f}\right)\cdots\mathbf{L}\left(\bw{0}{f}\right)\delta\bw{0}{f} = \mathbf{L}_k\delta\bw{0}{f} \label{Lk}\tag{8}
$$

(\ref{ww})に(\ref{Lk})を代入して転置を取ると

$$
\nabla J(\bw{0}{f}) = \sum_k\mathbf{L}_k^T\left(\bw{k}{f}-\bw{k}{o}\right) \label{nablaJ}\tag{9}
$$

が得られます。
$\mathbf{L}_k^T$を随伴演算子と呼びます。

$$
\mathbf{L}_k^T = \mathbf{L}\left(\bw{0}{f}\right)\mathbf{L}\left(\bw{1}{f}\right)\cdots\mathbf{L}\left(\bw{k-2}{f}\right)\mathbf{L}\left(\bw{k-1}{f}\right) \label{LT}\tag{10}
$$

随伴モデル
$$
\dbp{k-1}{f} = \mathbf{L}^T(\bw{k-1}{f})\dbp{k}{f} \label{adm}\tag{11}
$$
とすると随伴変数時刻$0$での解
$$
\dbp{0}{f} = \mathbf{L}_k^T\dbp{k}{f}　\label{adm0}\tag{12}
$$

はステップ $k$ からステップ $0$ まで逆方向に随伴演算子を作用させることにより得られます。
随伴変数を $\dbp{k}{f} = \bw{k}{f} - \bw{k}{o}$ にとり随伴演算子を作用させたものは (\ref{nablaJ}) の和の中身で，
コスト函数の勾配に対するタイムステップ$k$における$\bw{k}{f}-\bw{k}{o}$の寄与を表しています。

随伴モデルの計算は順方向の計算負荷と同程度です。勾配は要素一つずつに摂動を与えて求めることもできますが，要素数だけの積分が必要となるほか，精度の点で不利です。勾配が得られれば，数値最適化手法を適用して最小値を求めることができます。


### Lorenz-63の接線型モデルと随伴モデル

Lorenz-63モデルの接線型モデルと随伴モデルを求めます。接線型モデルは非線型モデルを基本場の周りに線型化することで得られます。線型項は変数を摂動に置き換えます。二つの変数が掛け合わされている非線型項は一方が摂動である二つの項に分かれます。

$$
\begin{align}
\dot{\delta X} &= -\sigma \delta X + \sigma \delta Y \\
\dot{\delta Y} &= -\delta XZ -X\delta Z + r\delta X - \delta Y \\
\dot{\delta Z} &= \delta XY X\delta Y - b\delta Z
\end{align}
$$

随伴モデルは，接線型モデルを3つの変数とその微分に作用する行列として表し，その転置を取ることで得られます。

$$
\begin{align}
\delta X^{\mathrm{a}} &= \delta X^\mathrm{a} - \sigma\dot{\delta X}^\mathrm{a} + (r-Z)\dot{\delta Y}^\mathrm{a} + Y\dot{\delta Z}^\mathrm{a} \\
\delta Y^{\mathrm{a}} &= \delta Y^{\mathrm{a}} + \sigma\dot{\delta X}^\mathrm{a} - \dot{\delta Y}^\mathrm{a} + X\dot{\delta Z}^\mathrm{a} \\
\delta Z^\mathrm{a} &= \delta Z^\mathrm{a} - X\dot{\delta Y}^\mathrm{a} - b\dot{\delta Z}^\mathrm{a}
\end{align}
$$

### Pythonコード

Lorenz-63の非線型モデル，接線型モデル，随伴モデルをPythonで書いてみましょう。
Prandtl数，Rayleigh数，アスペクト比はそれぞれ`p`, `r`, `b`で表されています。
`x, y, z = w`のように1次元配列を展開して式との対応がつきやすいように工夫しています。

In [1]:
def florenz(w, p, r, b):
    x, y, z = w
    dw = np.zeros_like(w)
    dw[0] =      -p * x + p * y 
    dw[1] = (r - z) * x -     y 
    dw[2] =           x * y     - b * z
    return dw

In [2]:
def tlorenz(wt, wb, p, r, b):
    dw = np.zeros_like(wt)
    xt, yt, zt = wt
    xb, yb, zb = wb
    dw[0] =       -p * xt +  p * yt 
    dw[1] = (r - zb) * xt -      yt - xb * zt
    dw[2] =       yb * xt + xb * yt -  b * zt
    return dw

In [3]:
def alorenz(wa, dwa, wb, p, r, b):
    xb, yb, zb = wb
    dxa, dya, dza = dwa
    wa[0] += -p * dxa + (r - zb) * dya + yb * dza
    wa[1] +=  p * dxa -            dya + xb * dza
    wa[2] +=          -       xb * dya -  b * dza
    dwa[:] = 0.0
    return wa, dwa

### ニューラルネットワーク